In [ ]:
"""
모델이 정확하더라도, inference time이 크면 안된다.
inference time의 최고기준이 필요
예) 카메라로 촬영시, inference time이 크다면, 프레임이 뚝뚝 끊겨 정확한 데이터가 안옴
적어야만 프레임끊김없이 스무스한 데이터를 확보할 수 있을 것이다.
이처럼, accuracy도 어느 정도 이상이며, inference time도 어느 정도 이하가 돼야 함

그렇다면, 아키텍처의 복잡도를 유지하면서 inference time을 낮추는 법이 있나?
이걸 연구한게 Model Qunatization
예) float32를 int8로 변경하는 등, 연산에 더 좋은 데이터를 바꾸기
이점- 큰 모델에서 accuracy drop이 발생하지 않고 변환이 잘 됨(모델이 작은 경우 약간 발생)

이를 개선하기 위해, quantization을, 데이터(Weight) 학습 동시에 실시.
float버전과 int버전을 동시에 학습시킴. accuracy 차이가 감소.
"""

In [4]:
!pip install -q tensorflow-model-optimization
!pip install flask-ngrok

In [5]:
!ls
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from sklearn.preprocessing import MinMaxScaler

sample_data


In [ ]:
!pip list | grep numpy
# pip list에서 받은 결과물을 grep(찾고), numpy가 포함된 걸 출력

In [10]:
from flask import Flask, render_template, request
from flask_ngrok import run_with_ngrok

#load data
X = pd.read_csv('X.csv')

with open('y.npy', 'rb') as f:
  y = np.load(f)

X = X[['OverallQual', 'GrLivArea', 'GarageCars', 'GarageArea', 'TotalBsmtSF', 
       '1stFlrSF', 'FullBath', 'LotShape_rank']]


x_min_max_scaler = MinMaxScaler()
y_min_max_scaler = MinMaxScaler()
x_min_max_scaler.fit(X)
y_min_max_scaler.fit(y) #X, y의 feature별 최대최소값을 스케일러가 기억

scaled_X = x_min_max_scaler.transform(X)
scaled_y = y_min_max_scaler.transform(y)

In [11]:
model = keras.Sequential(
      [
          keras.Input(shape=scaled_X.shape[-1]),
          layers.Dense(96, activation='relu'),
          layers.Dense(48, activation='relu'),
          layers.Dense(1)
      ]
  )

model.compile(loss="mse", optimizer="adam")

early_stopping_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=15)
model.fit(scaled_X, scaled_y, 
          batch_size=2, epochs=150, 
          callbacks=[early_stopping_callback], validation_split=0.05)

Epoch 1/150
694/694 [==============================] - 2s 2ms/step - loss: 0.0045 - val_loss: 0.0017
Epoch 2/150
694/694 [==============================] - 1s 2ms/step - loss: 0.0032 - val_loss: 0.0051
Epoch 3/150
694/694 [==============================] - 1s 2ms/step - loss: 0.0032 - val_loss: 0.0048
Epoch 4/150
694/694 [==============================] - 1s 2ms/step - loss: 0.0027 - val_loss: 0.0015
Epoch 5/150
694/694 [==============================] - 2s 2ms/step - loss: 0.0030 - val_loss: 0.0015
Epoch 6/150
694/694 [==============================] - 2s 2ms/step - loss: 0.0025 - val_loss: 0.0015
Epoch 7/150
694/694 [==============================] - 2s 2ms/step - loss: 0.0025 - val_loss: 0.0014
Epoch 8/150
694/694 [==============================] - 1s 2ms/step - loss: 0.0025 - val_loss: 0.0013
Epoch 9/150
694/694 [==============================] - 2s 2ms/step - loss: 0.0023 - val_loss: 0.0015
Epoch 10/150
694/694 [==============================] - 1s 2ms/step - loss: 0.0024 - val_lo

In [12]:
pred = model.predict(scaled_X[:1]) # 0 ~ 1
pred = y_min_max_scaler.inverse_transform(pred)

In [13]:
#앞서 학습시킨 모델(adam형 compile)을, 학습시키지 않고, 바로 모델을 저장하고 load해서 쓰고자할때
#학습된 모델을, h5라는 형태로 저장하고, load_model 통해 그대로 쓸 수 있음
model.save("mlp_v0.1.h5")

#load model
reconstructed_model = keras.models.load_model("mlp_v0.1.h5")

pred = reconstructed_model.predict(scaled_X[:1]) # 0 ~ 1
pred = y_min_max_scaler.inverse_transform(pred)

pred

array([[204978.22]], dtype=float32)

In [ ]:
#submit form이 위치한 곳을 알려줌(/content)
app = Flask(__name__, template_folder = '/content')
run_with_ngrok(app) #flask 앱 선언 후, copy 할 수 있도록 함(ngrok에 권한 줌)

def preprocess_data(data) :
  #TODO: 전처리. 8자리 정보를 np 딕셔너리 array로 변경시킨다(1,8짜리)
  #이 때, LotShpae는 str로 들어오므로, 이를 적절한 값으로 변환이 필요
  X = [] # <-- 각각 값을 넣을 배열
  for k, v in data.items() :
    if k == 'LotShape' :
      if v=='Reg' :
        X.append(4)
      elif v == 'IR1' :
        X.append(3)
      elif v == 'IR2':
        X.append(2)
      else :
        X.append(1)
    else :
      X.append(float(v))

  # X = [2, 5000, 2, ..., 3]
  X = np.array(X) #(8,)
  X = X.reshape((1, -1)) #(1,8)

  scaled_X = x_min_max_scaler.transform(X)
  print(scaled_X.shape)
  return scaled_X

@app.route("/")
def predict() :
  #return "<h1> This is your flask server</h1>"
  return render_template("submit_form.html")

#submit버튼 누르면 result가 호출됨(form action = "/result"에 의해)
@app.route("/result", methods=['POST'])
def result():
  #data 읽고, data전처리 후, Model prediction시켜 값 리턴
  #Salpeprice와 가장 상관관계 높은 10개의 것을 선택해 집값 유추하고자함
  #GrLivArea의 경우, X['GrLivArea'].min(), max(), median()을 통해 값 범위 보자

  #받아온 데이터를 읽어들이자
  data = request.form

  message = ""
  message +="<hl> House Price </hl>"

  for k, v in data.items():
    print(k,v)
    message += k+": " + v + "</br>" 
  
  #데이터 전처리
  X = preprocess_data(data) #user가 보낸 data를
  # X: (1,8)
  
  #pred = model.predict(X)
  pred = reconstructed_model.predict(X)
  pred = y_min_max_scaler.inverse_transform(pred)
  #이 때, pred shape는 (1,1).

  message += "</br>"
  message += "Predict price: " + str(pred[0][0]) 
  print(message)
  return message

app.run() #앱실행. 유저가 요청 보낼 함수를 여러 개 만들어두면, 더 복잡한 앱서버 생성이 가능
#지금까진, 코랩 터미널에서 요청 보내면, 내가 응답을 받게 됨. 
#ngrok에 대신 요청을 보내면, 코랩을 찾아서 응답을 카피해 나한테 보내줄 것. 이 떄, 난 외부사용자(edge)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://e9a4-35-189-162-164.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [20/Nov/2021 12:18:54] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [20/Nov/2021 12:18:55] "GET /favicon.ico HTTP/1.1" 404 -
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:446: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  "X does not have valid feature names, but"
127.0.0.1 - - [20/Nov/2021 12:18:56] "POST /result HTTP/1.1" 200 -


OverallQual 6
GrLivArea 1464
GarageArea 480
GaregeCars 2
TotalBsmtSF 991
1stFlrSF 1087
FullBath 2
LotShape IR1
(1, 8)
<hl> House Price </hl>OverallQual: 6</br>GrLivArea: 1464</br>GarageArea: 480</br>GaregeCars: 2</br>TotalBsmtSF: 991</br>1stFlrSF: 1087</br>FullBath: 2</br>LotShape: IR1</br></br>Predict price: 6635052.5


127.0.0.1 - - [20/Nov/2021 13:26:47] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [20/Nov/2021 13:26:48] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [20/Nov/2021 13:26:49] "GET / HTTP/1.1" 200 -
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:446: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  "X does not have valid feature names, but"
127.0.0.1 - - [20/Nov/2021 13:26:51] "POST /result HTTP/1.1" 200 -


OverallQual 6
GrLivArea 1464
GarageArea 480
GaregeCars 2
TotalBsmtSF 991
1stFlrSF 1087
FullBath 2
LotShape IR1
(1, 8)
<hl> House Price </hl>OverallQual: 6</br>GrLivArea: 1464</br>GarageArea: 480</br>GaregeCars: 2</br>TotalBsmtSF: 991</br>1stFlrSF: 1087</br>FullBath: 2</br>LotShape: IR1</br></br>Predict price: 6635052.5


127.0.0.1 - - [20/Nov/2021 13:26:52] "GET /result HTTP/1.1" 405 -
